In [1]:
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [2]:
file_path = Path('../data/raw/Ventas.csv' )# Asegúrate de que este archivo esté en la misma ruta que tu notebook o proporciona la ruta completa.

df = pl.read_csv(file_path.absolute(), encoding='windows-1252', separator=';')

df.head()

id_venta,producto,cantidad,precio_unitario,fecha_venta,region
i64,str,i64,str,str,str
1001,"""Snacks""",5530,""" 152,58 € ""","""30/11/2020""","""Norte"""
1002,"""Cárnicos""",994,""" 421,89 € ""","""28/01/2020""","""Centro"""
1003,"""Cereales""",6845,""" 205,70 € ""","""21/11/2020""","""Norte"""
1004,"""Frutas""",9806,""" 9,33 € ""","""02/10/2021""","""Sur"""
1005,"""Alimento infantil""",3633,""" 255,28 € ""","""12/11/2022""","""Norte"""


In [7]:
# valores nulos
if df.is_empty():
    print("El DataFrame está vacío. No hay columnas para analizar.")

# Crear una lista para almacenar los resultados de cada columna
resultados = []
total_filas = df.height

for column_name in df.columns:
    # Contar el número de valores nulos en la columna actual
    nulos_count = df[column_name].is_null().sum()

    # Calcular el porcentaje de valores nulos
    nulos_porcentaje = (nulos_count / total_filas) * 100 if total_filas > 0 else 0

    # Obtener el tipo de dato de la columna
    tipo_dato = df[column_name].dtype

    resultados.append({
        'Columna': column_name,
        'Nulos_Count': nulos_count,
        'Nulos_Porcentaje': nulos_porcentaje,
        'Tipo_Dato': str(tipo_dato) # Convertir el tipo de dato a string para el DataFrame final
    })

# Convertir la lista de diccionarios a un DataFrame de Polars
resumen_nulos = pl.DataFrame(resultados)

# Ordenar por porcentaje de nulos de forma descendente
resumen_nulos = resumen_nulos.sort(by='Nulos_Porcentaje', descending=True)
resumen_nulos

Columna,Nulos_Count,Nulos_Porcentaje,Tipo_Dato
str,i64,f64,str
"""region""",163,16.3,"""String"""
"""producto""",17,1.7,"""String"""
"""cantidad""",11,1.1,"""Int64"""
"""id_venta""",0,0.0,"""Int64"""
"""precio_unitario""",0,0.0,"""String"""
"""fecha_venta""",0,0.0,"""String"""


In [3]:
df_ventas_pl = df.with_columns(
    # 1. Conversión de tipos de datos
    # Convertir 'fecha_venta' a datetime
    pl.col('fecha_venta').str.strptime(pl.Date, format='%d/%m/%Y', strict=False).alias('fecha_venta'),
    # Limpiar y convertir 'precio_unitario' a numérico (float)
    pl.col('precio_unitario').str.replace(' €', '').str.replace(',', '.', literal=True).str.replace(' ', '', n=2).cast(pl.Float64).alias('precio_unitario'),
    # Convertir 'cantidad' a numérico
    pl.col('cantidad').cast(pl.Int64).alias('cantidad')
)

In [4]:
# 2. Manejo de valores nulos
# Para 'producto': Identificado en la muestra, una fila tenía 'producto' vacío.
# En Polars, `fill_null` o `fill_nan` se usan para nulos. Los strings vacíos no son nulos por defecto.
# Primero, reemplazar strings vacíos con nulos para poder usar fill_null
df_ventas_pl = df_ventas_pl.with_columns(
    pl.when(pl.col('producto') == '').then(pl.lit(None, pl.String)).otherwise(pl.col('producto')).alias('producto')
)
# Ahora llenar los nulos en 'producto'
df_ventas_pl = df_ventas_pl.with_columns(
    pl.col('producto').fill_null('Desconocido').alias('producto')
)

In [5]:
# Eliminar filas con nulos en columnas críticas después de la transformación

print("\n--- Manejo de valores nulos (Imputación por Media/Moda) ---")

# Columnas numéricas para imputar con la media
numeric_cols_to_impute = ['precio_unitario', 'cantidad']

for col in numeric_cols_to_impute:
    if df_ventas_pl[col].null_count() <= 0:
        continue # Si no hay nulos, saltar

    # Calcular la media de la columna
    mean_value = df_ventas_pl[col].mean()
    # Imputar los valores nulos con la media calculada
    df_ventas_pl = df_ventas_pl.with_columns(
        pl.col(col).fill_null(mean_value).alias(col)
    )
    print(f"Valores nulos en '{col}' imputados con la media ({mean_value:.2f}).")

# Columnas categóricas para imputar con la moda
categorical_cols_to_impute = ['region'] # Añade aquí otras columnas categóricas si es necesario

for col in categorical_cols_to_impute:
    if df_ventas_pl[col].null_count() <= 0:
        continue # Si no hay nulos, saltar

    # Calcular la moda (valor más frecuente) de la columna
    # Polars tiene un método `mode()`. Si hay múltiples modas, toma la primera.
    mode_value = df_ventas_pl.group_by(col).len().sort(by='len', descending=True).select(col).head(1).item()
    
    # Imputar los valores nulos con la moda calculada
    df_ventas_pl = df_ventas_pl.with_columns(
        pl.col(col).fill_null(mode_value).alias(col)
    )
    print(f"Valores nulos en '{col}' imputados con la moda ('{mode_value}').")



initial_rows_before_date_dropna = df_ventas_pl.shape[0]
df_ventas_pl = df_ventas_pl.drop_nulls(subset=['fecha_venta'])
rows_removed_date_nulls = initial_rows_before_date_dropna - df_ventas_pl.shape[0]

if rows_removed_date_nulls > 0:
    print(f"Se eliminaron {rows_removed_date_nulls} filas con valores nulos en 'fecha_venta' (después de intentar la conversión).")

print("\nValores nulos después de la imputación/eliminación final:")
print(df_ventas_pl.null_count()) # Usar .null_count() en Polars para ver los nulos por columna


--- Manejo de valores nulos (Imputación por Media/Moda) ---
Valores nulos en 'cantidad' imputados con la media (4975.12).
Valores nulos en 'region' imputados con la moda ('Sur').

Valores nulos después de la imputación/eliminación final:
shape: (1, 6)
┌──────────┬──────────┬──────────┬─────────────────┬─────────────┬────────┐
│ id_venta ┆ producto ┆ cantidad ┆ precio_unitario ┆ fecha_venta ┆ region │
│ ---      ┆ ---      ┆ ---      ┆ ---             ┆ ---         ┆ ---    │
│ u32      ┆ u32      ┆ u32      ┆ u32             ┆ u32         ┆ u32    │
╞══════════╪══════════╪══════════╪═════════════════╪═════════════╪════════╡
│ 0        ┆ 0        ┆ 0        ┆ 0               ┆ 0           ┆ 0      │
└──────────┴──────────┴──────────┴─────────────────┴─────────────┴────────┘


In [21]:
# 4. Tratamiento de valores atípicos (ej: cantidad = 0)
initial_rows_before_zero = df_ventas_pl.shape[0]
df_ventas_pl = df_ventas_pl.filter(pl.col('cantidad') > 0)
rows_removed_zero_qty = initial_rows_before_zero - df_ventas_pl.shape[0]
if rows_removed_zero_qty > 0:
    print(f"Se eliminaron {rows_removed_zero_qty} filas donde 'cantidad' era igual o menor a 0.")


Se eliminaron 9 filas donde 'cantidad' era igual o menor a 0.


In [23]:
# Guardar el DataFrame transformado a un archivo CSV para uso posterior
output_file_path = Path('../data/processed/Ventas_cleaned.parquet')
csv_output_file_path = Path('../data/processed/Ventas_cleaned.csv')
df_ventas_pl.write_parquet(output_file_path)
df_ventas_pl.write_csv(csv_output_file_path)
